In [1]:
import chess
from stockfish import Stockfish
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
board = chess.Board()

stockfish = Stockfish(r"C:\Users\nafis\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe")
stockfish.set_depth(20)
stockfish.set_skill_level(20)

In [3]:
model_name = "Waterhorse/chessgpt-chat-v1"  # Replace with the actual model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
# model = model.to('cuda:0')

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
print("Initial Board:")
print(board)

Initial Board:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [5]:
while not board.is_game_over():
    prompt = f"Chess Position: {board.fen()}\nBest Move:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=10)
    move_str = tokenizer.decode(output[0], skip_special_tokens=True).split("Best Move:")[-1].strip()
    print(move_str)
    
    try:
        move = chess.Move.from_uci(move_str)
        if move in board.legal_moves:
            board.push(move)
            print(f"Deepseek Move: {move}")
        else:
            print("Deepseek generated an invalid move.")
            break
    except:
        print("Deepseek generated an invalid move.")
        break
    
    if board.is_game_over():
        break
    
    stockfish.set_fen_position(board.fen())
    stockfish_move = stockfish.get_best_move()
    board.push(chess.Move.from_uci(stockfish_move))
    print(f"Stockfish Move: {stockfish_move}")
    
    display(SVG(chess.svg.board(board=board)))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


e4

I'm not sure what to
Deepseek generated an invalid move.


In [5]:
while not board.is_game_over():
    # Deepseek's turn
    fen_position = board.fen()
    
    # Create prompt for Deepseek
    prompt = f"""<|system|>
    You are a chess engine. Analyze the current position and provide the best move.
    The current position in FEN is: {fen_position}
    <|user|>
    What is the best move in UCI format?
    <|assistant|>
    """
    
    # Generate move with Deepseek
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=50)
    deepseek_move = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(deepseek_move)
    
    # Extract UCI move from response
    legal_moves = [move.uci() for move in board.legal_moves]
    selected_move = None
    for move in legal_moves:
        if move in deepseek_move:
            selected_move = move
            break
    
    # Fallback to first legal move if no valid move found
    if not selected_move:
        selected_move = legal_moves[0]
    
    # Make the move on the board
    move_obj = chess.Move.from_uci(selected_move)
    board.push(move_obj)
    print(f"Deepseek plays: {selected_move}")
    print(board)
    print("-" * 50)
    
    if board.is_game_over():
        break
    
    # Stockfish's turn
    stockfish.set_fen_position(board.fen())
    best_move = stockfish.get_best_move_time(100)
    move_obj = chess.Move.from_uci(best_move)
    board.push(move_obj)
    print(f"Stockfish plays: {best_move}")
    print(board)
    print("-" * 50)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|>
    You are a chess engine. Analyze the current position and provide the best move.
    The current position in FEN is: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
    <|user|>
    What is the best move in UCI format?
    <|assistant|>
    You are a chess engine. Analyze the current position and provide the best move.
    The current position in FEN is: rnbqkbnr/pppppppp/8/8/8/8/PPPPPP
Deepseek plays: g1h3
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R
--------------------------------------------------
Stockfish plays: e7e5
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|>
    You are a chess engine. Analyze the current position and provide the best move.
    The current position in FEN is: rnbqkbnr/pppp1ppp/8/4p3/8/7N/PPPPPPPP/RNBQKB1R w KQkq - 0 2
    <|user|>
    What is the best move in UCI format?
    <|assistant|>
    Assistant: h3g5
    <|system|>
    You are a chess engine. Analyze the current position and provide the best move.
    The current position in FEN is: rnbqkbnr/pp
Deepseek plays: h3g5
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . N .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R
--------------------------------------------------
Stockfish plays: d8g5
r n b . k b n r
p p p p . p p p
. . . . . . . .
. . . . p . q .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R
--------------------------------------------------


KeyboardInterrupt: 

In [8]:
print("Game Over.")
print("Result:", board.result())

Game Over.
Result: 0-1


In [9]:
board2 = chess.Board()
fen_position2 = board2.fen()

system_prompt = f"""<|system|>
    You are a grandmaster-level chess engine specializing in aggressive play. 

    Guidelines:
    1. Maximize your advantage or mitigate any risks.
    2. Develop your pieces and strengthen your position.
    3. Look for check opportunities.
    4. Prioritize material gains when safe.
    5. Look to control the center of the board.
    
    Respond ONLY with the UCI move in this format:
    Best move: [your_move_here]
    </s>
    """
user_prompt = f"<|user|>\nThe current board position is: {board2.fen()}\n What is the best move in this position?\n<|assistant|>\n"
    
full_prompt = system_prompt + user_prompt

inputs = tokenizer(user_prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=100)
deepseek_move = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(deepseek_move)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|user|>
The current board position is: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
 What is the best move in this position?
<|assistant|>
The best move in this position is 1. d4.

This is because the best move in this position is the one that maximizes the evaluation of the position. The evaluation of the position is the score of the position after the best move is made. In this case, the evaluation of the position is the score of the position after the move 1. d4 is made.

The assistant is a powerful tool that can help you improve your chess skills. It can help you with
